# Notebook 3: Case Triage & Response Assistance

**User Story**: US3 — Case Triage & Response Assistance  
**Persona**: Customer Service Representative (CSR)  

Present a case to the Service Agent for priority recommendation, category,
draft response grounded in Knowledge Base articles, and source citations.
Write-back requires explicit CSR confirmation.

In [ ]:
# Cell 2: Environment + Auth Setup
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
env_path = Path("../.env")
if env_path.exists():
    load_dotenv(env_path)

# Verify required environment variables
required_vars = [
    "AZURE_AI_PROJECT_ENDPOINT",
    "AZURE_OPENAI_DEPLOYMENT",
    "SF_INSTANCE_URL",
    "SF_ACCESS_TOKEN",
]
missing = [v for v in required_vars if not os.environ.get(v)]
if missing:
    raise OSError(f"Missing required environment variables: {missing}")

print("Environment configured successfully.")
print(f"  Azure AI endpoint: {os.environ['AZURE_AI_PROJECT_ENDPOINT'][:40]}...")
print(f"  Salesforce instance: {os.environ['SF_INSTANCE_URL']}")

In [ ]:
# Cell 3: Start MCP Servers + Configure Connections
import subprocess
import sys
import time

from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import MCPTool
from azure.identity import DefaultAzureCredential

project_root = os.path.abspath("..")

# Start salesforce-crm MCP server (SSE)
CRM_PORT = 8102
crm_process = subprocess.Popen(
    [sys.executable, "-m", "mcp_servers.salesforce_crm.server"],
    env={**os.environ, "MCP_TRANSPORT": "sse", "FASTMCP_PORT": str(CRM_PORT)},
    cwd=project_root,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)

# Start salesforce-knowledge MCP server (SSE)
KB_PORT = 8103
kb_process = subprocess.Popen(
    [sys.executable, "-m", "mcp_servers.salesforce_knowledge.server"],
    env={**os.environ, "MCP_TRANSPORT": "sse", "FASTMCP_PORT": str(KB_PORT)},
    cwd=project_root,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)

time.sleep(3)

for name, proc in [("CRM", crm_process), ("Knowledge", kb_process)]:
    if proc.poll() is not None:
        stderr = proc.stderr.read().decode() if proc.stderr else ""
        raise RuntimeError(f"{name} MCP server failed to start: {stderr}")

print(f"✅ CRM MCP server started (PID: {crm_process.pid}) at http://127.0.0.1:{CRM_PORT}/sse")
print(f"✅ Knowledge MCP server started (PID: {kb_process.pid}) at http://127.0.0.1:{KB_PORT}/sse")

# Create AI Project client
project_client = AIProjectClient(
    endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

# Create MCP tools
crm_mcp = MCPTool(server_label="salesforce-crm", server_url=f"http://127.0.0.1:{CRM_PORT}/sse")
knowledge_mcp = MCPTool(server_label="salesforce-knowledge", server_url=f"http://127.0.0.1:{KB_PORT}/sse")

print(f"✅ MCP tools configured: {crm_mcp.server_label}, {knowledge_mcp.server_label}")

In [ ]:
# Cell 4: Create OpenAI Client + Load System Prompt
from pathlib import Path as _Path

# Load service agent system prompt
system_prompt_path = _Path("../agents/service/system_prompt.md")
system_prompt = system_prompt_path.read_text(encoding="utf-8")

MODEL = os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt-4o")
client = project_client.get_openai_client()

print(f"OpenAI client ready — model: {MODEL}")
print("Tools: salesforce-crm (cases, accounts), salesforce-knowledge (articles)")

In [ ]:
# Cell 5: Triage request — "Triage case #12345 and suggest a response"
from IPython.display import Markdown, display

triage_message = (
    "Triage case #12345 and suggest a response. "
    "Include priority recommendation, category, "
    "a draft response grounded in Knowledge Base articles, "
    "and cite the KB article sources."
)

response = client.responses.create(
    model=MODEL,
    instructions=system_prompt,
    input=triage_message,
    tools=[crm_mcp, knowledge_mcp],
)

print(f"Response ID: {response.id}")
for item in response.output:
    if item.type == "message":
        for content in item.content:
            if content.type == "output_text":
                display(Markdown(content.text))

In [ ]:
# Cell 6: User confirms write-back — "Apply this triage"

confirm_message = (
    "Apply this triage. Update the case priority and category as recommended, "
    "and add the draft response as an internal comment."
)

confirm_response = client.responses.create(
    model=MODEL,
    instructions=system_prompt,
    input=confirm_message,
    tools=[crm_mcp, knowledge_mcp],
    previous_response_id=response.id,
)

print(f"Response ID: {confirm_response.id}")
for item in confirm_response.output:
    if item.type == "message":
        for content in item.content:
            if content.type == "output_text":
                display(Markdown(content.text))

In [ ]:
# Cell 7: Verify update in Salesforce — removed old Cell 8 (merged here)

verify_message = "Show me the current details of case #12345 to confirm the update was applied."

verify_response = client.responses.create(
    model=MODEL,
    instructions=system_prompt,
    input=verify_message,
    tools=[crm_mcp, knowledge_mcp],
    previous_response_id=confirm_response.id,
)

print(f"Response ID: {verify_response.id}")
for item in verify_response.output:
    if item.type == "message":
        for content in item.content:
            if content.type == "output_text":
                display(Markdown(content.text))

In [ ]:
# Cell 8: Cleanup — terminate MCP servers

crm_process.terminate()
crm_process.wait(timeout=5)
print(f"CRM MCP server terminated (PID: {crm_process.pid}).")

kb_process.terminate()
kb_process.wait(timeout=5)
print(f"Knowledge MCP server terminated (PID: {kb_process.pid}).")

print("\nCase triage session complete.")